## **Build a Web Research Agent**

### **Introduction**

In the previous excercise, you learned the basics of the Tavily API. Now, let's take it a step further: we'll combine that knowledge with LLMs to unlock real value. In this tutorial, you'll learn how to build a web research agent that can search, extract, crawl and reason over live web data.

You already explored the various parameter configurations available for each Tavily API endpoint. With the official `Tavily-LangChain integration`, our agent can automatically set these parameters - like `time_range` for search or specific crawl `instructions` - based on the context and requirements of each task. This dynamic, LLM-powered configuration is powerful in agentic systems.

By the end of this lesson, you'll know how to:
- Seamlessly connect foundation models to the web for up-to-date research
- Build a react-style web agent
- Dynamically configure search, extract and crawl parameters the Tavily-LangChain integration

### **Getting Started** 

In [1]:
# install dependencies
!pip install -U tavily-python langchain-google-genai langchain langchain-tavily langgraph python-dotenv --quiet

### **Setting Up Tavily Client**

In [4]:
import os
from dotenv import load_dotenv
from tavily import TavilyClient

load_dotenv()

tavily_api_key = os.getenv("TAVILY_API_KEY")
gg_api_key = os.getenv("GOOGLE_API_KEY")

if tavily_api_key is None or gg_api_key is None:
    print("Please set API Key")
else:
    tavily_client = TavilyClient(api_key=tavily_api_key)
    print("Setup succesfully")

Setup succesfully


Let's define the following modular tools with the Tavily-LangChain integration:
1. **Search** the web for relevant information
2. **Extract** content from specific web pages
3. **Crawl** entire websites

In [5]:
# Define the set of web tools our agent will use to interact with the Tavily API
from langchain_tavily import TavilySearch, TavilyCrawl, TavilyExtract

search = TavilySearch(max_results=5, topic="general")
extract = TavilyExtract(extract_depth="advanced")
crawl = TavilyCrawl()

Initialize the LLM Agent

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    api_key = gg_api_key
)

# test setup
response = llm.invoke("Hi, how are you?")
print(response.content)

I'm doing well, thank you for asking!

How are you today?
